In [1]:
from json import dumps

from pymongo import MongoClient
from IPython.display import Markdown, display, Image
from scipy.linalg import expm
from ipyparallel import Client, require

In [2]:
def print_logs(log,  level='INFO'):
    printmd('##### ' + log.name)
    for d in log.find({'level':level}):
        printmd(dumps(d['message']))
        
def printmd(string):
    ''' thanks http://stackoverflow.com/questions/32026727/format-output-of-code-cell-with-markdown '''
    display(Markdown(string))

In [3]:
@require('numpy')
def _is_Q_ok(Q):
    """Tests whether a square matrix is a valid transition rate matrix"""
    n = Q.shape[0]
    if not numpy.allclose(Q.imag, 0.):
        return False
    offd = Q * (1. - numpy.eye(n))
    if not numpy.allclose(offd[offd<0.], 0.):
        return False
    one = numpy.ones(n)
    if not numpy.allclose(Q.dot(one), 0.):
        return False
    return True

@require('numpy', 'numpy.linalg')
def is_generator_unique(Q):
    """Conservatively tests whether a transition rate matrix uniquely yields
    its transition probability matrix"""
    
    assert _is_Q_ok(Q), 'Q must be a transition rate matrix'
    
    e, V = numpy.linalg.eig(Q)
    invV = numpy.linalg.inv(V)
    n = len(e)
    
    # Q must be diagonalisable
    if not numpy.allclose(V.dot(numpy.diag(e)).dot(invV), Q):
        return False
        
    # Find the conjugate pairs and make the problem smaller
    pairs = set()
    for i in range(n):
        real_multiplicity = 1
        for j in range(n):
            if i == j:
                continue
                
            # All eigenvalues must be unique
            if numpy.isclose(e[i], e[j]):
                return False
            
            if numpy.isclose(e[i].real, e[j].real):
                real_multiplicity += 1
                pairs.add(frozenset([i,j]))
                
        # Real parts can have multiplicity of at most two
        if real_multiplicity > 2:
            return False
            
    # Real distinct eigenvalues, all good
    if len(pairs) == 0:
        return True
            
    # Calculate equivalent Q basis
    real_basis = []
    imag_basis = []
    ix = numpy.logical_not(numpy.eye(n, dtype=bool))
    for i, j in pairs:
        diff = numpy.outer(V[:,i], invV[i]) - numpy.outer(V[:,j], invV[j])
        base = 2.*numpy.pi*complex(0.,1.)*diff
        #newQ = Q + base
        #assert not allclose(Q, newQ)
        #assert allclose(expm(Q), expm(newQ))
        base = numpy.matrix(base[ix]).T
        real_basis.append(base.real)
        imag_basis.append(base.imag)
        
    # Constraints Mk 1
    # Q is real
    A_eq = numpy.hstack(imag_basis)
    b_eq = numpy.zeros(n*(n-1))
    
    # Q has no fewer zeros
    mask = numpy.isclose(Q[ix], 0.)
    
    A_eq = numpy.vstack([A_eq, numpy.hstack(real_basis)[mask]])
    b_eq = numpy.hstack([b_eq, numpy.zeros(mask.sum())])
    
    # A_eq * x == b_eq
    
    # assert allclose(A_eq.dot(zeros(len(pairs))), b_eq), 'wtf1'
    
    if numpy.allclose(A_eq, 0.):
        return False
    
    return A_eq.shape[1] == numpy.linalg.matrix_rank(A_eq)
    
    
#   # off-diagonal Q is non-negative
#   A_ub = -hstack(real_basis)
#   b_ub = Q[ix]
#   
#   # A_ub * x <= b_ub
#   
#   assert (A_ub.dot(zeros(len(pairs))) <= b_ub).all(), 'wtf2'
#   
#   A_eq = array(A_eq)
#   A_ub = array(A_ub)

#   dim = len(pairs)
#   lower = zeros(dim)
#   upper = zeros(dim)
#   for j in range(dim):
#       c = zeros(dim)
#       c[j] = 1.
#       result = linprog(c, A_ub, b_ub, A_eq, b_eq, bounds=[(-1, 1)]*dim, options=dict(tol=1e-9))
#       print result
#       if not result.success:
#           return False
#       lower[j] = result.fun
#       c[j] = -1.
#       result = linprog(c, A_ub, b_ub, A_eq, b_eq, bounds=[(-1, 1)]*dim, options=dict(tol=1e-9))
#       print result
#       if not result.success:
#           return False
#       upper[j] = result.fun
#       
#   return lower, upper

def test_Qs(database):
    @require('codon.nest', 'codon.ml', 'traceback')
    def check_lf(doc):
        try:
            gc = codon.ml.get_genetic_code(doc['gc'])
            model = lambda: codon.ml.GNC(gc=gc)
            lf = codon.nest.inflate_likelihood_function(doc['lf'], model)
            for tip in lf.tree.tips():
                tip = tip.Name
                Q = lf.getRateMatrixForEdge(tip).asarray()
                t = lf.getParamValue('length', edge=tip)
                if not is_generator_unique(Q*t):
                    return doc['_id'], tip
        except (KeyboardInterrupt, SystemExit):
            raise
        except:
            return doc['_id'], traceback.format_exc()
    
    docs = list(database.GNC.find())
    rc = Client()
    rc[:].push(dict(is_generator_unique=is_generator_unique,
                    _is_Q_ok=_is_Q_ok))
    failures = rc[:].map_async(check_lf, docs)
    return [f for f in failures if f is not None]

In [4]:
client = MongoClient()

In [5]:
datasets = ['hum_xen_fug', 'mammals', 'ants', 'introns']
ds_names = {'mammals' : 'Mammals',
            'ants' : 'Ants',
            'hum_xen_fug' : 'Human/Xenopus/Fugu',
            'introns' : 'Primate Introns'}
for dataset in datasets:
    printmd('#### ' + ds_names[dataset])
    print_logs(getattr(getattr(client, dataset), 'GNC.log'))

#### Human/Xenopus/Fugu

##### GNC.log

{"function": "ml.ml", "start_over": true, "log_level": "DEBUG", "no_mpi_main_loop": false, "kwargs_file": "../config/GNC.json", "input_collection": "hum_xen_fug.data", "output_collection": "hum_xen_fug.GNC", "output_collections_file": null, "output_collections": ["hum_xen_fug.GNC"], "db_host": "r2081", "input_collections_file": null, "kwargs": {"model": "GNC"}, "log_name": "log", "input_collections": ["hum_xen_fug.data"]}

{"mong": "0.0.10-dev", "monglog": "0.0.1-dev", "map_collection": "0.0.8-dev", "masterslave": "0.0.10-dev", "ml": "0.0.11-dev"}

#### Mammals

...

{"mong": "0.0.10-dev", "monglog": "0.0.1-dev", "map_collection": "0.0.8-dev", "masterslave": "0.0.10-dev", "ml": "0.0.11-dev"}

#### Primate Introns

##### GNC.log

{"function": "ml.ml", "start_over": true, "log_level": "DEBUG", "no_mpi_main_loop": false, "kwargs_file": "../config/GNC_no_stop.json", "input_collection": "introns.data", "output_collection": "introns.GNC", "output_collections_file": null, "output_collections": ["introns.GNC"], "db_host": "r2081", "input_collections_file": null, "kwargs": {"model": "GNC", "gc": "FFLLSSSSYYZOCCUWLLLLPPPPHHQQRRRRIIIMTTTTNNKKSSRRVVVVAAAADDEEGGGG"}, "log_name": "log", "input_collections": ["introns.data"]}

{"mong": "0.0.10-dev", "monglog": "0.0.1-dev", "map_collection": "0.0.8-dev", "masterslave": "0.0.10-dev", "ml": "0.0.11-dev"}

In [6]:
failures = {}
for dataset in datasets:
    failures[dataset] = test_Qs(getattr(client, dataset))
    printmd('##### Done ' + dataset)

##### Done hum_xen_fug

##### Done mammals

##### Done ants

##### Done introns

In [7]:
for dataset in failures:
    print dataset, len(failures[dataset])

introns 2139
mammals 95
hum_xen_fug 27
ants 11


In [8]:
import json
with open('uniqueness-failures.json', 'w') as uout:
    json.dump(failures, uout)